In [1]:
from cval import CVal

config_file = "../config.toml"
env_file = "../.env"

cval = CVal.init(
    config_file=config_file,
    env_file=env_file
)

/Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


[06/27/24 15:40:46] INFO     Ingestion engine initialized.                                          ]8;id=42218;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=746320;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py#42\42]8;;\

[06/27/24 15:40:47] INFO     Retrieval engine initialized.                                          ]8;id=26187;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=420928;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#29\29]8;;\

                    INFO     Ingestion engine initialized.                                          ]8;id=445201;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=460573;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     GPT (gpt-4o-2024-05-13) generator initialized.                         ]8;id=716887;file:///Users/sebastiansimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=749224;file:///Users/sebastiansimon/GitHub/cval/src/generator.py#47\47]8;;\

                    INFO     CVal initialized.                                                          ]8;id=271526;file:///Users/sebastiansimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=40342;file:///Users/sebastiansimon/GitHub/cval/src/cval.py#107\107]8;;\

In [2]:
import pandas as pd

eval_data_file = "../data/eval/dependencies_without_rules.csv"
df = pd.read_csv(eval_data_file, sep=";")
df_sample = df[:1]
df_sample

,link_str,project,option_name,option_value,option_file,option_type,option_technology,dependent_option_name,dependent_option_value,dependent_option_file,dependent_option_type,dependent_option_technology,pre_rating,rating,Notes
0,mall::::mall-admin/src/main/resources/applicat...,mall,spring.datasource.druid.stat-view-servlet.logi...,druid,mall-admin/src/main/resources/application-dev.yml,ConfigType.PASSWORD,spring,spring.datasource.druid.stat-view-servlet.logi...,druid,mall-search/src/main/resources/application-pro...,ConfigType.PASSWORD,spring,False,True,"borderline case, false if each service has it..."


In [3]:
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from dotenv import dotenv_values
import os

config = dotenv_values(dotenv_path="../.env")
os.environ['OPENAI_API_KEY'] = config["OPENAI_KEY"]


from deepeval.models.base_model import DeepEvalBaseLLM
from llama_index.llms.ollama import Ollama

model = Ollama(model="llama3:8b", temperature=0.0)

class LlamaModel(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()
        return model.complete(prompt).text

    async def a_generate(self, prompt: str) -> str:
        self.generate(prompt)

    def get_model_name(self):
        return "Custom Llama3 model"


llama_model = LlamaModel(model=model)


answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=0.7,
    model=llama_model,
    include_reason=True
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model=llama_model,
    include_reason=True
)

context_relevancy_metric = ContextualRelevancyMetric(
    threshold=0.7,
    model=llama_model,
    include_reason=True
)

/Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/deepeval/__init__.py:42: UserWarning: You are using deepeval version 0.21.61, however version 0.21.62 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [4]:
from typing import List
import pandas as pd
import json

def measure_validation_metrics(outputs: List, df_baseline: pd.DataFrame):

    true_positives = []
    true_negatives = []
    false_positives = []
    false_negatives = []
    accuracy = []

    for response, baseline in zip(outputs, df_baseline.to_dict("records")):

        response_dict = json.loads(response.response)

        if baseline["rating"] in ("True") and response_dict["isDependency"]:
            accuracy.append(1)
            true_positives.append(1)
        if baseline["rating"] in ("False") and not response_dict["isDependency"]:
            accuracy.append(1)
            true_negatives.append(1)
        if baseline["rating"] in ("True") and not response_dict["isDependency"]:
            accuracy.append(0)
            false_negatives.append(1)
        if baseline["rating"] in ("False") and response_dict["isDependency"]:
            accuracy.append(0)
            false_positives.append(1)


    #precision = sum(true_positives)/(sum(true_positives)+sum(false_positives))
    #recall = sum(true_positives)/(sum(true_positives)+sum(false_negatives))
    #f1_score = 2 * (precision * recall) / (precision + recall)

    print("Accuracy", sum(accuracy)/len(accuracy))
    print("TP", sum(true_positives))
    print("FP", sum(false_positives))
    print("TN", sum(true_negatives))
    print("FN", sum(false_negatives))
    #print("Precision", precision)
    #print("Recall", recall)
    #print("F1 Score: ", f1_score)

    return {
        "true_positives": sum(true_positives),
        "false_positives": sum(false_positives),
        "true_negatives": sum(true_negatives),
        "false_negatives": sum(false_negatives),
        "accuracy": sum(accuracy)/len(accuracy)
    }

In [5]:
from typing import List
import pandas as pd


def measure_llm_netrics(outputs: List):
    faithfulness_scores = []
    answer_relevancy_scores = []
    context_relevancy_scores = []

    for response in outputs:
        
        test_case = LLMTestCase(
            input=response.input,
            actual_output=response.response,
            retrieval_context=[source_node.node.get_content() for source_node in response.source_nodes]
        )

        context_relevancy_metric.measure(test_case)
        faithfulness_metric.measure(test_case)
        answer_relevancy_metric.measure(test_case)

        context_relevancy_scores.append(context_relevancy_metric.score)
        faithfulness_scores.append(faithfulness_metric.score)
        answer_relevancy_scores.append(answer_relevancy_metric.score)       

    context_relevance = sum(context_relevancy_scores)/len(context_relevancy_scores)
    answer_relevance = sum(answer_relevancy_scores)/len(answer_relevancy_scores)
    faithfulness = sum(faithfulness_scores)/len(faithfulness_scores)
            
    print("Context Relevancy: ", context_relevancy_scores, context_relevance)
    print("Answer Relevancy: ", answer_relevancy_scores, answer_relevance)
    print("Faithfulness: ", faithfulness_scores, faithfulness)

    return {
        "answer_relevance": answer_relevance,
        "faithfulness": faithfulness,
        "context_relevance": context_relevance
    }

In [6]:
from data import Dependency
import mlflow
import datetime

index_name = "so-posts"
exp_name = "dependencies_without_rules"
date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

mlflow.set_experiment(experiment_name=exp_name)

with mlflow.start_run(run_name=f"{exp_name}_{date_time}"):

    outputs = []

    for x in df_sample.to_dict("records"):
        dependency = Dependency(
            project=x["project"],
            option_name=x["option_name"],
            option_value=x["option_value"],
            option_type=x["option_type"].split(".")[-1],
            option_file=x["option_file"],
            option_technology=x["option_technology"],
            dependent_option_name=x["dependent_option_name"],
            dependent_option_value=x["dependent_option_value"],
            dependent_option_type=x["dependent_option_type"].split(".")[-1],
            dependent_option_file=x["dependent_option_file"],
            dependent_option_technology=x["dependent_option_technology"]
        )

        response = cval.query(
            dependency=dependency,
            index_name=index_name
        )

        outputs.append(response)

    llm_metrics = measure_llm_netrics(outputs=outputs)
    validation_metrics =measure_validation_metrics(outputs=outputs, df_baseline=df_sample)

    results = [x.response for x in outputs]
    inputs = [x.input for x in outputs]
    rating = df_sample["rating"]
    df_results = pd.DataFrame().from_dict(data={"outputs": results, "isDependency": rating})
    df_results.to_csv(f"../data/results/test_{index_name}.csv", index=False)
    mlflow.log_table(data={"inputs": inputs, "outputs": results}, artifact_file="results.json")

    

[06/27/24 15:40:48] INFO     Select index: so-posts.                                                ]8;id=1201;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=854563;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#47\47]8;;\

[06/27/24 15:40:51] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=766043;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=269112;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=554028;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=875851;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

[06/27/24 15:40:52] INFO     Len of retrieved nodes of query 0: 5                                  ]8;id=343237;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=216982;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#174\174]8;;\

[06/27/24 15:40:53] INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=557577;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=770108;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

...

[06/27/24 15:42:11] INFO     HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200   ]8;id=560708;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=160157;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

[06/27/24 15:42:14] INFO     HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200   ]8;id=364909;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=975128;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

[06/27/24 15:42:17] INFO     HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200   ]8;id=628440;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=522787;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

                    ERROR    Giving up get_scores(...) after 5 tries (AttributeError: 'NoneType'     ]8;id=927358;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/backoff/_common.py\_common.py]8;;\:]8;id=381839;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/backoff/_common.py#120\120]8;;\
                             object has no attribute 'find')                                                       

AttributeError: 'NoneType' object has no attribute 'find'